In [31]:
import numpy

In [59]:
def make_matrix(color):
    """
    Generates a transformation matrix for the specified color.
    Inputs:
        color: string with exactly one of the following values:
               'red', 'blue', 'green', or 'none'
    Returns:
        matrix: a transformation matrix corresponding to
                deficiency in that color
    """
    # You do not need to understand exactly how this function works.
    if color == 'red':
        c = [[.567, .433, 0], [.558, .442, 0], [0, .242, .758]]
    elif color == 'green':
        c = [[0.625, 0.375, 0], [0.7, 0.3, 0], [0, 0.142, 0.858]]
    elif color == 'blue':
        c = [[.95, 0.05, 0], [0, 0.433, 0.567], [0, 0.475, .525]]
    elif color == 'none':
        c = [[1, 0., 0], [0, 1, 0.], [0, 0., 1]]
    return c


def matrix_multiply(m1, m2):
    """
    Multiplies the input matrices.
    Inputs:
        m1,m2: the input matrices
    Returns:
        result: matrix product of m1 and m2
        in a list of floats
    """

    product = numpy.matmul(m1, m2)
    if type(product) == numpy.int64:
        return float(product)
    else:
        result = list(product)
        return result

In [60]:
from PIL import Image, ImageFont, ImageDraw

def img_to_pix(filename):
    """
    Takes a filename (must be inputted as a string
    with proper file attachment ex: .jpg, .png)
    and converts to a list of representing pixels.

    For RGB images, each pixel is a tuple containing (R,G,B) values.
    For BW images, each pixel is an integer.

    # Note: Don't worry about determining if an image is RGB or BW.
            The PIL library functions you use will return the 
            correct pixel values for either image mode.

    Returns the list of pixels.

    Inputs:
        filename: string representing an image file, such as 'lenna.jpg'
        returns: list of pixel values 
                 in form (R,G,B) such as [(0,0,0),(255,255,255),(38,29,58)...] for RGB image
                 in form L such as [60,66,72...] for BW image
    """
    img = Image.open(filename)
    return list(img.getdata())

In [61]:
a = img_to_pix("image_15.png")
print(len(a))

73170


In [62]:
def pix_to_img(pixels_list, size, mode):
    """
    Creates an Image object from a inputted set of RGB tuples.

    Inputs:
        pixels_list: a list of pixels such as the output of
                img_to_pixels.
        size: a tuple of (width,height) representing
              the dimensions of the desired image. Assume
              that size is a valid input such that
              size[0] * size[1] == len(pixels).
        mode: 'RGB' or 'L' to indicate an RGB image or a 
              BW image, respectively
    returns:
        img: Image object made from list of pixels
    """
    img = Image.new(mode,size)
    img.putdata(pixels_list)
    return img
    
    pass

In [63]:
x = pix_to_img(a, (270, 271), "RGB")
x.show()

In [77]:
def filter(pixels_list, color):
    """
    pixels_list: a list of pixels in RGB form, such as
            [(0,0,0),(255,255,255),(38,29,58)...]
    color: 'red', 'blue', 'green', or 'none', must be a string representing 
           the color deficiency that is being simulated.
    returns: list of pixels in same format as earlier functions,
    transformed by matrix multiplication
    """
    return_list = []
    m2 =  make_matrix(color)
    print(m2)
    for i in pixels_list:
        b = matrix_multiply(i, m2)  # Matrix multiplication returns floats

        # Clamp and convert to int properly
        b_clamped = tuple(max(0, min(255, int(round(val)))) for val in b)

        return_list.append(b_clamped)

    return return_list

In [70]:
b = filter(a, "red")

c = pix_to_img(b, (270, 271), "RGB")
c.show()

[[0.567, 0.433, 0], [0.558, 0.442, 0], [0, 0.242, 0.758]]


In [78]:
def extract_end_bits(num_end_bits, pixel):
    """
    Extracts the last num_end_bits of each value of a given pixel.

    example for BW pixel:
        num_end_bits = 5
        pixel = 214

        214 in binary is 11010110. 
        The last 5 bits of 11010110 are 10110.
                              ^^^^^
        The integer representation of 10110 is 22, so we return 22.

    example for RBG pixel:
        num_end_bits = 2
        pixel = (214, 17, 8)

        last 3 bits of 214 = 110 --> 6
        last 3 bits of 17 = 001 --> 1
        last 3 bits of 8 = 000 --> 0

        so we return (6,1,0)

    Inputs:
        num_end_bits: the number of end bits to extract
        pixel: an integer between 0 and 255, or a tuple of RGB values between 0 and 255

    Returns:
        The num_end_bits of pixel, as an integer (BW) or tuple of integers (RGB).
    """
    if type(pixel) == int:
        return pixel%2**num_end_bits
    else:
        return tuple(i%2**num_end_bits for i in list(pixel))
    pass


In [83]:
extract_end_bits(3, (1235, 12, 1))

(3, 4, 1)

In [86]:
a = Image.open("hidden1.bmp")
a.show()


In [100]:
def reveal_bw_image(filename):
    """
    Extracts the single LSB for each pixel in the BW input image. 
    Inputs:
        filename: string, input BW file to be processed
    Returns:
        result: an Image object containing the hidden image
    """
    return_list = []
    pixels = img_to_pix(filename)
    for i in pixels:
        return_list.append(extract_end_bits(1, i))
    return_list_final = [i*255 for i in return_list]
    size = Image.open(filename).size
    mode = "L"
    return pix_to_img(return_list_final, size, mode)

In [113]:
def reveal_color_image(filename):
    """
    Extracts the 3 LSBs for each pixel in the RGB input image. 
    Inputs:
        filename: string, input RGB file to be processed
    Returns:
        result: an Image object containing the hidden image
    """
    return_list = []
    pixels = img_to_pix(filename)
    for i in pixels:
        return_list.append(extract_end_bits(3, i))
    return_list_final = []
    for i in return_list:
        return_list_final.append(tuple([int(round((j*255)/7))for j in list(i)]))
            
    size = Image.open(filename).size
    mode = "RGB"
    return pix_to_img(return_list_final, size, mode)
    pass


In [114]:
y = reveal_color_image("hidden2.bmp")
y.show()

In [93]:
b = [1,2,3]
a = [i*2 for i in b]
a

[2, 4, 6]

In [95]:
a = (1,2,3)*2
a

(1, 2, 3, 1, 2, 3)

In [117]:
a = "shubham gupta"
a.split("a")

['shubh', 'm gupt', '']